# Übungsaufgaben DQL

Der folgende Block enthält Übungsaufgaben zu den DQL-Kapiteln.

## SELECT

Löse diese Aufgaben, nachdem du den SELECT-Block bearbeitet hast.

**1.1** Zeige alle verfügbaren (aktiven) Kunden an.

In [ ]:
USE WideWorldImporters;
SELECT * FROM sales.Customers;

**1.2** Zeige die Vor- und Nachnamen aller Benutzer an

In [ ]:
USE WideWorldImporters;
SELECT FullName FROM Application.People;

**1.3** Welche Liefermethoden kennt WWI?

In [ ]:
USE WideWorldImporters;
SELECT DeliveryMethodName FROM Application.DeliveryMethods;

**1.4** Erstelle eine Abfrage auf die Tabelle _Sales.Customers,_ welche dir folgende Felder liefert: CustomerId, CustomerName, Address (Bestehend als konkatiniertes Feld aus den Feldern PostalAddressLine1, PostalAddressLine2 und PostalPostalCode).

In [ ]:
USE WideWorldImporters;
SELECT CustomerID, CustomerName,
PostalAddressLine1 + ' ' + PostalAddressLine2 + ' ' + PostalPostalCode
AS 'Address'
FROM Sales.Customers;

## WHERE-Bedingungen

Führe diese Aufgaben aus, nachdem du den Block "WHERE-Bedingungen" bearbeitet hast.

**2.1** Wie viele Mitarbeiter zählt WWI?

In [ ]:
USE WideWorldImporters;
SELECT COUNT(*) AS 'Anzahl Mitarbeiter' FROM Application.People
WHERE IsEmployee = 1;

**2.2** Wie viele Mitarbeiter sind Verkäufer?

In [ ]:
USE WideWorldImporters;
SELECT COUNT(*) AS 'Anzahl Verkäufer' FROM Application.People
WHERE IsSalesperson = 1;

**2.3**: Gebe alle Städte der Städte-Tabelle aus, welche mehr als 10'000 Einwohner haben.

In [ ]:
USE WideWorldImporters;
SELECT DISTINCT CityName, LatestRecordedPopulation 
FROM Application.Cities
WHERE LatestRecordedPopulation > 10000;

**2.4** Liste alle Städte auf, welche mit "Ab" beginnen.

In [ ]:
USE WideWorldImporters;
SELECT DISTINCT CityName
FROM Application.Cities
WHERE CityName LIKE 'Ab%'

## JOIN

Führe diese Aufgaben aus, nachdem du den JOIN-Block bearbeitet hast.

**3.1** Erstelle eine Abfrage, welche die Tabelle _Customers_ mit der Tabelle _Orders_ vereint. Es sollen nur Kunden ausgegeben werden, welche etwas bestellt haben. Die Abfrage soll mindestens die Felder _CustomerId_ und _OrderId_ enthalten.

In [ ]:
-- Hinweis: INNER JOIN ist default und entspricht JOIN
-- Durch INNER JOIN werden nur Kunden ausgegeben, die was bestellt haben (Schnittmenge)
USE WideWorldImporters;
SELECT c.CustomerID, c.CustomerName, o.OrderID 
FROM Sales.Customers AS c
JOIN Sales.Orders AS o
ON c.CustomerID = o.CustomerID;


**3.2** Welche Kunden sind in der Datenbank erfasst, welche über keine Bestellungen verfügen? Liste explizit die Kunden auf, welche über keine Bestellung verfügen.

In [ ]:
USE WideWorldImporters;

-- mit Mengenoperation EXCEPT
USE WideWorldImporters;
SELECT CustomerID AS 'Kunden IDs ohne Bestellung' FROM Sales.customers
EXCEPT
SELECT CustomerID FROM Sales.orders;

-- oder mit LEFT JOIN
SELECT DISTINCT c.CustomerID AS 'Kunden IDs ohne Bestellung' -- in der Lösung fehlt DISTINCT
FROM Sales.Customers c LEFT JOIN Sales.Orders o ON c.CustomerID = o.CustomerID
WHERE o.CustomerID IS NULL;

-- oder mit Subquery
SELECT c.customerid AS 'Kunden IDs ohne Bestellung'
FROM Sales.customers c
WHERE c.customerid NOT IN (SELECT DISTINCT customerid FROM Sales.Orders) -- schöner mit DISTINCT


**3.3** Gebe für jeden Kunden die dazugehörige Rechnungsadresse aus. (Tipp: Das Feld _BillToCustomerId_ zeigt dir, wer der Rechnungsempfänger ist).

In [ ]:
/*Hier sind die Lösungen falsch. Meiner Ansicht nach müsst die Frage
zur Musterlösung so heissen:
Gib alle Kunden inkl. Addresse aus, welche als Rechnungsempfänger erfasst sind.*/

-- Richtig ist mit SELF JOIN
USE WideWorldImporters;
SELECT c.CustomerName 'Kunde', 
cb.CustomerName + ' ' + cb.DeliveryAddressLine1 + ' ' + cb.DeliveryAddressLine2
+ ', ' + cb.PostalPostalCode + ' ' + ci.CityName 'Rechnungsadresse'
FROM Sales.Customers c JOIN Sales.Customers cb
ON c.BillToCustomerID = cb.CustomerID
JOIN Application.Cities ci ON cb.PostalCityID = ci.CityID; -- das hier braucht es noch, um City-Name anzugeben

## CASE

Führe diese Aufgaben aus, nachdem du den CASE-Block bearbeitet hast.

**4.1** Erstelle folgende Abfrage: Liste Personen auf, die entweder Mitarbeiter, Verkäufer oder Kunde sind. Das Resultat soll dabei folgende Felder enthalten: Vollständiger Name, Telefon-Nummer, Fax-Nummer, Rolle. In der Rolle sollen folgende Werte abgebildet werden: Mitarbeiter, wenn eine Peron ein Mitarbeiter ist, Verkäufer, wenn eine Person Verkäufer ist, Kunde, wenn die Person weder Verkäufer noch Mitarbeiter ist. Einträge ohne Telefon unx Fax-Nummer sollen herausgefiltert werden.

In [ ]:
USE WideWorldImporters;
SELECT FullName, PhoneNumber, FaxNumber,
CASE
WHEN IsSystemUser = 1 AND IsEmployee = 1 AND isSalesperson =1 THEN 'USER / MITARBEITER / VERKÄUFER' 
WHEN IsSystemUser = 0 AND IsEmployee = 1 AND isSalesperson =1 THEN 'MITARBEITER / VERKÄUFER'
WHEN IsSystemUser = 1 AND IsEmployee = 0 AND isSalesperson =1 THEN 'USER / VERKÄUFER'
WHEN IsSystemUser = 1 AND IsEmployee = 1 AND isSalesperson =0 THEN 'USER / MITARBEITER'
WHEN IsSystemUser = 1 AND IsEmployee = 0 AND isSalesperson =0 THEN 'USER' 
WHEN IsSystemUser = 0 AND IsEmployee = 0 AND isSalesperson =1 THEN 'VERKÄUFER' 
WHEN IsSystemUser = 0 AND IsEmployee = 1 AND isSalesperson =0 THEN 'MITARBEITER' 
ELSE 'KUNDE '
END AS 'ROLLE'
FROM Application.People
WHERE PhoneNumber IS NOT NULL OR FaxNumber IS NOT NULL

## Gruppierungen

Führe diese Aufgaben aus, nachdem du den Gruppierungs-Block bearbeitet hast.

**5.1** Finde die zehn teuersten Bestellungen, welche WWI entgegennehmen durfte

In [ ]:
USE WideWorldImporters;
SELECT TOP 10 ol.OrderID, SUM(ol.Quantity * ol.UnitPrice) 'Betrag' -- Rechnet für jede Bestellposition den Betrag aus und summiert alle Positionen zusammen
FROM Sales.OrderLines ol
GROUP BY ol.OrderID
ORDER BY 'Betrag' DESC;


**5.2** Zu welchen Kunden gehören die zehn teuersten Bestellungen?

In [ ]:
USE WideWorldImporters;
SELECT TOP(10) ol.OrderID, c.CustomerName, SUM(Quantity * UnitPrice) AS 'Betrag' 
FROM Sales.OrderLines AS ol
JOIN Sales.Orders AS o ON ol.OrderID = o.OrderID
JOIN Sales.Customers AS c ON o.CustomerID = c.CustomerID
GROUP BY ol.OrderID, c.CustomerName
ORDER BY 'Betrag' DESC;

-- Oder Lösung mit Unterabfrage: Query von Aufgabe 5.1 als Unterabfrage verwenden und darauf JOIN machen
-- Dieser Ansatz ist in Lösungen, ist jedoch viel komplexer. Besser meine Lösung oben verwenden
SELECT o.OrderID, c.CustomerName, sub.Betrag
FROM Sales.Customers c JOIN Sales.Orders o ON c.CustomerID = o.CustomerID
JOIN
(SELECT TOP 10 ol.OrderID, SUM(ol.Quantity * ol.UnitPrice) 'Betrag' -- Rechnet für jede Bestellposition den Betrag aus und summiert alle Positionen zusammen
FROM Sales.OrderLines ol
GROUP BY ol.OrderID
ORDER BY 'Betrag' DESC) AS sub ON o.OrderID = sub.OrderID ORDER BY sub.Betrag DESC; 



**5.3** Welches sind die zehn Umsatzstärksten Kunden von WWI?

In [12]:
USE WideWorldImporters;

SELECT TOP 10 c.CustomerName, SUM(ol.Quantity * ol.UnitPrice) 'Total'
FROM Sales.Customers c JOIN Sales.Orders o ON c.CustomerID = o.CustomerID
JOIN Sales.OrderLines ol ON o.OrderID = ol.OrderID
GROUP BY c.CustomerName
ORDER BY 'Total' DESC;

**5.4** Wie viele Provinzen haben die einzelnen hinterlegten Lieferländer von WWI? Gebe dazu die Anzahl Provinzen gruppiert nach Land aus.

In [13]:
USE WideWorldImporters;

SELECT CountryName, COUNT(s.StateProvinceID) AS 'Anzahl Provinzen'
FROM Application.Countries c JOIN Application.StateProvinces s
ON c.CountryID = s.CountryID
GROUP BY c.CountryName;

-- Hinweis: Offenbar sind nur für USA Provinzen erfasst. Überprüfung mit LEFT JOIN

**5.5** Finde alle Bestellungen, welche mehr oder genau 5 Bestellpositionen haben.

In [15]:
USE WideWorldImporters;

SELECT o.OrderID, COUNT(OrderLineID) 'Anzahl Bestellpositionen' 
FROM Sales.Orders o JOIN Sales.OrderLines ol
ON o.OrderID = ol.OrderID
GROUP BY o.OrderID
HAVING COUNT(OrderLineID) >= 5;

## Unterabfragen

Führe diese Aufgaben aus, nachdem du den Unterabfragen-Block bearbeitet hast.

**6.1** Welche Kundenkategorien kannte WWI über die gesamte Laufbahn? Liste alle namentlich auf.

In [ ]:
-- Frage kann unterschiedlich verstanden werden

-- 1) Welche Kundenkategorien gibt es schon seit Beginn und sind nicht terminiert?
USE WideWorldImporters;

SELECT cc.CustomerCategoryName FROM Sales.CustomerCategories cc
WHERE cc.ValidFrom = (SELECT MIN(cc.ValidFrom) FROM Sales.CustomerCategories cc)
AND cc.ValidTo = (SELECT MAX(cc.ValidTo) FROM Sales.CustomerCategories cc);

-- 2a) Welche Kundenkategorien wurden von erfassten Kunden schon verwendet?
SELECT CustomerCategoryName FROM Sales.CustomerCategories
WHERE CustomerCategoryID IN (SELECT DISTINCT CustomerCategoryID FROM Sales.Customers)

-- 2b) Dasselbe noch mit JOIN anstatt Unterabfrage
SELECT DISTINCT cc.CustomerCategoryName
FROM Sales.CustomerCategories cc RIGHT JOIN Sales.Customers c 
ON cc.CustomerCategoryID = c.CustomerCategoryID;

## Spezialaufgaben

Führe die folgenden Aufgaben aus, nachdem du die anderen Blöcke fertig bearbeitet hast

**7.1** Erstelle eine Gesamtübersicht über die WWI für das Jahr 2013, die folgende Informationen enthält: Anzahl Kunden, Anzahl Bestellungen im Jahre 2013, Bestellvolumen Jan. - Mär. 2013, Bestellvolumen Mär. - Jun. 2013, Bestellvolumen Jul. - Sep. 2013, Bestellvolumen Okt - Dez. 2013.

In [ ]:
-- Deutlich schöner als in Lösung:

USE WideWorldImporters;

SELECT 
    (SELECT COUNT(CustomerID) FROM SALES.Customers) AS 'Anzahl Kunden',
    (SELECT COUNT(OrderID) FROM SALES.Orders WHERE YEAR(OrderDate) = 2013) AS 'Anzahl Bestellungen im Jahre 2013',
    (SELECT COUNT(OrderID) FROM SALES.Orders WHERE OrderDate BETWEEN '2013-01-01' AND '2013-03-31') AS 'Bestellvolumen Jan. - Mär. 2013',
    (SELECT COUNT(OrderID) FROM SALES.Orders WHERE OrderDate BETWEEN '2013-04-01' AND '2013-06-30') AS 'Bestellvolumen April - Jun. 2013',
    (SELECT COUNT(OrderID) FROM SALES.Orders WHERE OrderDate BETWEEN '2013-07-01' AND '2013-09-30') AS 'Bestellvolumen Jul. - Sep. 2013',
    (SELECT COUNT(OrderID) FROM SALES.Orders WHERE OrderDate BETWEEN '2013-10-01' AND '2013-12-31') AS 'Bestellvolumen Okt - Dez. 2013';


**7.2** Ermittle das gesamte Bestellvolumen für die Jahre 2013 bis 2016. Dabei soll für jedes Jahr eine eigene Spalte verwendet werden.

In [ ]:
USE WideWorldImporters;

SELECT 
    SUM(CASE WHEN YEAR(o.OrderDate) = 2013 THEN ol.Quantity * ol.UnitPrice ELSE 0 END) AS UMSATZ_2013,
    SUM(CASE WHEN YEAR(o.OrderDate) = 2014 THEN ol.Quantity * ol.UnitPrice ELSE 0 END) AS UMSATZ_2014,
    SUM(CASE WHEN YEAR(o.OrderDate) = 2015 THEN ol.Quantity * ol.UnitPrice ELSE 0 END) AS UMSATZ_2015,
    SUM(CASE WHEN YEAR(o.OrderDate) = 2016 THEN ol.Quantity * ol.UnitPrice ELSE 0 END) AS UMSATZ_2016
FROM 
    Sales.OrderLines ol
JOIN 
    Sales.Orders o ON ol.OrderID = o.OrderID;
